In [20]:
# common library
import os
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import argparse
import json
import cv2

# torch library
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models

# import my file
from lib.data_utils import test

1-10是simulator的数据集，11-14是Intraoperative的数据集

In [34]:

# base path 
base_path = 'Traindata/'
# simulator data set
simulator_num = [i+1 for i in range(10)]
s_tmp = '1'
simulator_path = base_path + 'aicm'+s_tmp+'/VID000_0'
# intraoperative data set
intra_num = [11, 12, 13, 14]
i_tmp = '11'
intra_path = 'aicm' + i_tmp

images_path = simulator_path + '/images'
labels_path = simulator_path + '/point_labels'

print(images_path)
print(labels_path)

Traindata/aicm1/VID000_0/images
Traindata/aicm1/VID000_0/point_labels


In [46]:
images_name = os.listdir(images_path)
labels_name = os.listdir(labels_path)
file_name = []
for i in images_name:
    file_name.append(i.split(".")[0])
#     print(i.split(".")[0])
# print(file_name)
# print(images_name)
# print(labels_name)




In [11]:
test()

I am in utils
